In [1]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

In [2]:
%cd /content/drive/MyDrive/NLP_Project/

/content/drive/MyDrive/NLP_Project


In [ ]:
# !wget https://www.gutenberg.org/files/1661/1661-0.txt

--2021-11-09 17:08:05--  https://www.gutenberg.org/files/1661/1661-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607430 (593K) [text/plain]
Saving to: ‘1661-0.txt’

1661-0.txt          100%[===================>] 593.19K   637KB/s    in 0.9s    

2021-11-09 17:08:12 (637 KB/s) - ‘1661-0.txt’ saved [607430/607430]



In [3]:
path = '/content/drive/MyDrive/NLP_Project/1661-0.txt'
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 581533


In [4]:
len(text.split())

107560

In [5]:
import numpy as np
len(np.unique(text.split()))

14533

In [4]:
text = text[:180000]

In [23]:
!head /content/drive/MyDrive/NLP_Project/1661-0.txt

﻿The Project Gutenberg eBook of The Adventures of Sherlock Holmes, by Arthur Conan Doyle

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at
www.gutenberg.org. If you are not located in the United States, you
will have to check the laws of the country where you are located before
using this eBook.



In [5]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [6]:
import pickle

# saving
with open('/content/drive/MyDrive/NLP_Project/tokenizer_RNN.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [8]:
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['the', 'project', 'gutenberg', 'ebook', 'of']
the


In [9]:
WORD_LENGTH = 5

In [10]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [11]:
print(X[0][0])

[False False False ... False False False]


In [14]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [16]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True)

Epoch 1/20
250/250 [==============================] - 51s 197ms/step - loss: 5.8948 - accuracy: 0.1027 - val_loss: 5.8752 - val_accuracy: 0.1111
Epoch 2/20
250/250 [==============================] - 48s 191ms/step - loss: 5.3831 - accuracy: 0.1408 - val_loss: 5.8358 - val_accuracy: 0.1212
Epoch 3/20
250/250 [==============================] - 48s 192ms/step - loss: 5.0426 - accuracy: 0.1814 - val_loss: 6.0266 - val_accuracy: 0.1147
Epoch 4/20
250/250 [==============================] - 48s 193ms/step - loss: 4.7334 - accuracy: 0.2382 - val_loss: 6.4212 - val_accuracy: 0.0969
Epoch 5/20
250/250 [==============================] - 48s 193ms/step - loss: 4.4403 - accuracy: 0.3079 - val_loss: 6.6033 - val_accuracy: 0.0951
Epoch 6/20
250/250 [==============================] - 48s 191ms/step - loss: 4.0922 - accuracy: 0.3807 - val_loss: 6.8158 - val_accuracy: 0.0826
Epoch 7/20
250/250 [==============================] - 47s 190ms/step - loss: 3.7499 - accuracy: 0.4521 - val_loss: 6.9139 - val_ac

In [17]:
model.save('keras_next_word_model.h5')
# pickle.dump(history, open("history.p", "wb"))
# model = load_model('keras_next_word_model.h5')
# history = pickle.load(open("history.p", "rb"))

In [14]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x
prepare_input("It is not a lack".lower())

it
is
not
a
lack


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [15]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [16]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

In [25]:
q =  "Project Gutenberg License included this"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  Project Gutenberg License included this
Sequence:  project gutenberg license included this
project
gutenberg
license
included
this
next possible words:  ['this', 'with', 'in', 'for', 'well']
